<h3>Nama kelompok :
    <li>Fransiskus Riswan Indra Simbolon
    <li>Exelindo Yeremia
    <li>Annisa Ramadhani
    <li>Wafiq Muharnisa Haspin
    <li>Arza Muhammad Prihandoyo
    <li>Yusuf Natanael

In [16]:
# Import Libraries
import json
import nltk
import time
import random
import string
import pickle
import numpy as np
import pandas as pd
from gtts import gTTS
from io import BytesIO
import tensorflow as tf
import IPython.display as ipd
import speech_recognition as sr 
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Model
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Flatten, Dense, GlobalMaxPool1D

In [17]:
# Package sentence tokenizer
nltk.download('punkt')
# Package lemmatization
nltk.download('wordnet')
# Package multilingual wordnet data
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/wan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/wan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/wan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [18]:
# Importing the dataset
with open('museum.json') as content:
  data1 = json.load(content)

# Mendapatkan semua data ke dalam list
tags = []
inputs = []
responses = {}
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])
    for pattern in intent['patterns']:
      w = nltk.word_tokenize(pattern)
      words.extend(w)
      documents.append((w, intent['tag']))
      # add to our classes list
      if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Konversi data json ke dalam dataframe
data = pd.DataFrame({"patterns":inputs, "tags":tags})


In [19]:
data['patterns'] = data['patterns'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['patterns'] = data['patterns'].apply(lambda wrd: ''.join(wrd))

In [20]:
# Lematisasi atau Lemmatization
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

print (len(words), "unique lemmatized words", words)

27 unique lemmatized words ['afternoon', 'apa', 'bangunan', 'bro', 'dari', 'deli', 'didirikan', 'filosofi', 'hai', 'hallo', 'halo', 'hei', 'hi', 'hy', 'itu', 'kapan', 'kawan', 'malam', 'morning', 'musebot', 'museum', 'pagi', 'serdang', 'si', 'siang', 'siapa', 'sore']


In [21]:
# sort classes
classes = sorted(list(set(classes)))
print (len(classes), "classes", classes)

4 classes ['MuseBot', 'filosofi', 'greeting', 'kapan']


In [22]:
# documents = combination between patterns and intents
print (len(documents), "documents")

237 documents


In [23]:
# Tokenize the data (Tokenisasi Data)
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['patterns'])
train = tokenizer.texts_to_sequences(data['patterns'])
train

[[7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [1, 22, 5],
 [23, 5],
 [1, 6, 24, 25, 2, 3, 4],
 [1, 6, 2, 3, 4],
 [26, 27, 2, 3, 4],
 []]

In [24]:
# Apply padding 
x_train = pad_sequences(train)
print(x_train) # Padding Sequences

[[ 0  0  0  0  0  0  7]
 [ 0  0  0  0  0  0  8]
 [ 0  0  0  0  0  0  9]
 [ 0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0 11]
 [ 0  0  0  0  0  0 12]
 [ 0  0  0  0  0  0 13]
 [ 0  0  0  0  0  0 14]
 [ 0  0  0  0  0  0 15]
 [ 0  0  0  0  0  0 16]
 [ 0  0  0  0  0  0 17]
 [ 0  0  0  0  0  0 18]
 [ 0  0  0  0  0  0 19]
 [ 0  0  0  0  0  0 20]
 [ 0  0  0  0  0  0 21]
 [ 0  0  0  0  1 22  5]
 [ 0  0  0  0  0 23  5]
 [ 1  6 24 25  2  3  4]
 [ 0  0  1  6  2  3  4]
 [ 0  0 26 27  2  3  4]
 [ 0  0  0  0  0  0  0]]


In [25]:
# Encoding the outputs 
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])
print(y_train)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 1 1 3 3]


In [26]:
input_shape = x_train.shape[1]
print(input_shape)

7


In [29]:
# define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ", vocabulary)

# output length
output_length = le.acclasses_.shape[2]
print("output length: ", output_length)

number of unique words :  27


AttributeError: 'LabelEncoder' object has no attribute 'acclasses_'